# 1.데이터 시스템 접속 및 데이터 추출 - [실습 시스템] 기상청 기온 데이터

In [ ]:
from selenium import webdriver
import pyautogui
from datetime import datetime, timedelta
import clipboard #파일경로 한글 입력에 필요

#기상 데이터(기온 데이터) 웹페이지 로딩
driver = webdriver.Ie('../2.web/IEDriverServer')
driver.get('https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70')
driver.maximize_window()
driver.implicitly_wait(20) #드라이버가 로딩되는데 10초까지 대기, 빨리 로딩되면 다음 코드 진행

#자료 검색 조건을 입력하기 위해 자료구분을 이미지로 검색하여 클릭, 이후 탭키로 이동(사전에 이미지로 캡처-저장)
pyautogui.sleep(1)
div_b = pyautogui.locateCenterOnScreen('div_b.png',confidence=0.9) #grayscale=True
pyautogui.click(div_b)

#기간 시작일, 종료일 설정
pyautogui.sleep(0.5)
start_date = datetime.now()-timedelta(weeks=4)
end_date = datetime.now()-timedelta(days=1)
start_date = start_date.strftime('%Y%m%d')
end_date = end_date.strftime('%Y%m%d')

#탭키로 이동하여 기간 입력
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.write(start_date) 
pyautogui.press('tab')
pyautogui.press('tab')
pyautogui.write(end_date)
pyautogui.press('enter')

#검색 버튼 클릭
pyautogui.sleep(0.5)
search_b = pyautogui.locateCenterOnScreen('search_b.png',confidence=0.8)
pyautogui.click(search_b)

#엑셀 다운 버튼 클릭
pyautogui.sleep(0.5)
excel_b = pyautogui.locateCenterOnScreen('excel_b.png',confidence=0.8)
pyautogui.click(excel_b)

#다른 이름으로 저장하기 위한 버튼 클릭
pyautogui.sleep(1)
name_save_b = pyautogui.locateCenterOnScreen('name_save_b.png',confidence=0.8)
pyautogui.click(name_save_b)
pyautogui.press('a') #다른 이름으로 저장 단축키 입력

#파일명에 경로와 함께 현재 일시를 포함하여 저장
now_date = datetime.now().strftime('%Y%m%d')
file_name = r'D:\작업방\python_RPA\4.training\weather_raw_data_' + now_date + '.xls'
clipboard.copy(file_name)
pyautogui.sleep(1)
print(file_name)
pyautogui.hotkey('ctrl','v')
pyautogui.sleep(0.5)
pyautogui.hotkey('alt','s')

pyautogui.sleep(1)
driver.quit() #드라이버 닫기
pyautogui.sleep(3)

# 2.엑셀파일 형식 변환 : xls --> xlsx

In [17]:
import pyautogui
from datetime import datetime, timedelta
import clipboard #파일경로 한글 입력에 필요

now_date = datetime.now().strftime('%Y%m%d')
file_name = r'D:\작업방\python_RPA\4.training\weather_raw_data_' + now_date + '.xls'
clipboard.copy(file_name)

#엑셀 파일 실행
pyautogui.hotkey('win','r') #프로그램 실행 단축키
pyautogui.sleep(0.5)
pyautogui.hotkey('ctrl','v')
pyautogui.press('enter')

#엑셀 창 최대화
pyautogui.sleep(15) #엑셀 실행하는 대기 시간을 충분히 줌
w = pyautogui.getWindowsWithTitle('weather_raw_data') #윈도우 타이틀과 같은 창 정보는 수집하여, 리스트로 리턴

if w[0].isActive == False:
    w[0].activate() #첫번째 윈도우 활성화
    
if w[0].isMaximized == False:
    w[0].maximize()

#다른 이름으로 저장하기
pyautogui.hotkey('alt','f') #메뉴-파일
pyautogui.press('a') #다른 이름으로 저장
pyautogui.press('o') #찾아보기

#파일형식 지정(xlsx)해서 저장
file_name = r'D:\작업방\python_RPA\4.training\weather_raw_data_' + now_date + '.xlsx'
clipboard.copy(file_name)
pyautogui.sleep(3)
pyautogui.hotkey('ctrl','v')
pyautogui.press('tab')
pyautogui.press('down')
pyautogui.press('home')
pyautogui.press('enter')
pyautogui.press('enter')

pyautogui.hotkey('alt', 'f4') #엑셀 창 닫기
pyautogui.sleep(3)

# 3.데이터 가공

In [24]:
import win32com.client

excel = win32com.client.Dispatch('Excel.Application')
excel.Visible = True
wb = excel.Workbooks.Open(file_name)
ws = wb.Activesheet

ws.Range('1:7').Delete() #상단의 정보가 있는 행 삭제
ws.Range('B:B').Delete() #지점 열 삭제
ws.Range('A:A').ColumnWidth = 15 #날자 열 너비 변경

wb.Save()
excel.Quit()
